In [17]:
#install required packages (run once)
!pip install --quiet pandas plotly ipywidgets
# (If you plan to build a Dash app later, also install: pip install dash jupyter-dash)
print("Packages installed (or already present).")


Packages installed (or already present).


In [19]:
#imports and load dataset
import pandas as pd
import plotly.express as px
from ipywidgets import widgets
from IPython.display import display, clear_output

DATA_PATH = "OnlineRetail.csv"   # <-- your uploaded file
df = pd.read_csv(DATA_PATH, encoding='unicode_escape')

print("Loaded dataset with rows,cols:", df.shape)
df.head()


Loaded dataset with rows,cols: (541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [28]:
#robust cleaning & preprocessing
# Helper to find the actual column name given candidates
def find_col(candidates, cols):
    for c in candidates:
        if c in cols:
            return c
    # try case-insensitive match
    lowcols = {col.lower():col for col in cols}
    for c in candidates:
        if c.lower() in lowcols:
            return lowcols[c.lower()]
    return None

cols = list(df.columns)

product_col = find_col(['Description','Product','description','product','ItemDescription','StockCode'], cols)
date_col    = find_col(['InvoiceDate','Invoice Date','Date','invoice_date','Invoice_Date'], cols)
qty_col     = find_col(['Quantity','quantity','Qty','QTY'], cols)
price_col   = find_col(['UnitPrice','Unit Price','unitprice','Price','Unit_Price'], cols)
invoice_col = find_col(['InvoiceNo','InvoiceNo','Invoice','Invoice No'], cols)
country_col = find_col(['Country','country'], cols)
customer_col= find_col(['CustomerID','Customer Id','Customer_ID','customerid'], cols)

print("Detected columns mapping:")
print(f" product_col: {product_col}")
print(f" date_col   : {date_col}")
print(f" qty_col    : {qty_col}")
print(f" price_col  : {price_col}")
print(f" invoice_col: {invoice_col}")
print(f" country_col: {country_col}")
print(f" customer_col: {customer_col}")

# Basic checks
if product_col is None:
    raise ValueError("Couldn't detect product/description column. Please check column names.")
if date_col is None:
    raise ValueError("Couldn't detect date column (InvoiceDate). Please check column names.")
if qty_col is None or price_col is None:
    raise ValueError("Couldn't detect Quantity or UnitPrice columns. Please check column names.")

# Work on a copy
df_clean = df.copy()

# Convert numeric columns safely
df_clean[qty_col] = pd.to_numeric(df_clean[qty_col], errors='coerce')
df_clean[price_col] = pd.to_numeric(df_clean[price_col], errors='coerce')

# Parse dates
df_clean[date_col] = pd.to_datetime(df_clean[date_col], errors='coerce')

# Drop rows missing critical info
df_clean = df_clean.dropna(subset=[product_col, date_col, qty_col, price_col])

# Remove cancellations if invoice column exists (usually starts with 'C')
if invoice_col:
    try:
        df_clean = df_clean[~df_clean[invoice_col].astype(str).str.startswith('C', na=False)]
    except Exception:
        pass

# Keep only positive quantities
df_clean = df_clean[df_clean[qty_col] > 0]

# Create unified columns
df_clean = df_clean.rename(columns={product_col: 'Product', date_col: 'InvoiceDate', qty_col: 'Quantity', price_col: 'UnitPrice'})
if country_col:
    df_clean = df_clean.rename(columns={country_col: 'Country'})
if invoice_col:
    df_clean = df_clean.rename(columns={invoice_col: 'InvoiceNo'})
if customer_col:
    df_clean = df_clean.rename(columns={customer_col: 'CustomerID'})

# Compute Sales, Date, Hour, Month
df_clean['Sales'] = df_clean['Quantity'] * df_clean['UnitPrice']
df_clean['Date'] = df_clean['InvoiceDate'].dt.date
df_clean['Hour'] = df_clean['InvoiceDate'].dt.hour
df_clean['Month'] = df_clean['InvoiceDate'].dt.to_period('M').astype(str)

print("After cleaning, rows:", df_clean.shape[0])
display(df_clean[['Product','InvoiceDate','Quantity','UnitPrice','Sales']].head())
# Replace original df pointer with cleaned
df = df_clean


Detected columns mapping:
 product_col: Product
 date_col   : InvoiceDate
 qty_col    : Quantity
 price_col  : UnitPrice
 invoice_col: InvoiceNo
 country_col: Country
 customer_col: CustomerID
After cleaning, rows: 397924


,Product,InvoiceDate,Quantity,UnitPrice,Sales
0,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00,6,2.55,15.30
1,WHITE METAL LANTERN,2010-12-01 08:26:00,6,3.39,20.34
2,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00,8,2.75,22.00
3,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00,6,3.39,20.34
4,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00,6,3.39,20.34


In [29]:
#Key Insights (Top countries, top products, sales by hour, monthly sales)
# Top 10 Countries by Sales (if Country exists)
if 'Country' in df.columns:
    top_countries = df.groupby('Country', as_index=False)['Sales'].sum().sort_values('Sales', ascending=False).head(10)
else:
    top_countries = pd.DataFrame(columns=['Country','Sales'])
    print("Country column not found; top_countries empty.")

# Top 10 Products by Sales
top_products = df.groupby('Product', as_index=False)['Sales'].sum().sort_values('Sales', ascending=False).head(10)

# Sales by hour (0-23)
sales_by_hour = df.groupby('Hour', as_index=False)['Sales'].sum().sort_values('Hour')

# Monthly sales
monthly_sales = df.groupby('Month', as_index=False)['Sales'].sum().sort_values('Month')

# Display
print("Top 10 Countries by Sales:")
display(top_countries)
print("\nTop 10 Products by Sales:")
display(top_products)
print("\nSales by Hour (0-23):")
display(sales_by_hour)
print("\nMonthly Sales:")
display(monthly_sales)

# Quick preview plots
fig_top_c = px.bar(top_countries, x='Country', y='Sales', title='Top 10 Countries by Sales') if not top_countries.empty else None
fig_top_p = px.bar(top_products, x='Product', y='Sales', title='Top 10 Products by Sales')
fig_hour  = px.bar(sales_by_hour, x='Hour', y='Sales', title='Sales by Hour (0-23)')

if fig_top_c:
    fig_top_c.show()
fig_top_p.show()
fig_hour.show()


Top 10 Countries by Sales:


,Country,Sales
35,United Kingdom,7308391.554
23,Netherlands,285446.340
10,EIRE,265545.900
14,Germany,228867.140
13,France,209024.050
0,Australia,138521.310
30,Spain,61577.110
32,Switzerland,56443.950
3,Belgium,41196.340
31,Sweden,38378.330



Top 10 Products by Sales:


,Product,Sales
2310,"PAPER CRAFT , LITTLE BIRDIE",168469.60
2756,REGENCY CAKESTAND 3 TIER,142592.95
3687,WHITE HANGING HEART T-LIGHT HOLDER,100448.15
1751,JUMBO BAG RED RETROSPOT,85220.78
1981,MEDIUM CERAMIC TOP STORAGE JAR,81416.73
2599,POSTAGE,77803.96
2335,PARTY BUNTING,68844.33
206,ASSORTED COLOUR BIRD ORNAMENT,56580.34
2119,Manual,53779.93
2644,RABBIT NIGHT LIGHT,51346.20



Sales by Hour (0-23):


,Hour,Sales
0,6,4.250
1,7,31059.210
2,8,282115.630
3,9,842605.171
4,10,1261192.571
5,11,1104558.750
6,12,1378571.480
7,13,1173264.750
8,14,995629.371
9,15,966191.750



Monthly Sales:


,Month,Sales
0,2010-12,572713.890
1,2011-01,569445.040
2,2011-02,447137.350
3,2011-03,595500.760
4,2011-04,469200.361
5,2011-05,678594.560
6,2011-06,661213.690
7,2011-07,600091.011
8,2011-08,645343.900
9,2011-09,952838.382


In [30]:
#dropdown options (keeps product list reasonable)
country_options = ['All']
if 'Country' in df.columns:
    country_options += sorted(df['Country'].dropna().unique().tolist())

TOP_N_PRODUCTS = 50  # change to larger if you want full list (may be slow)
top_products_list = df.groupby('Product')['Sales'].sum().sort_values(ascending=False).head(TOP_N_PRODUCTS).index.tolist()
product_options = ['All'] + top_products_list

print("Country options count:", len(country_options))
print("Using top N products for dropdown (N={}): {}".format(TOP_N_PRODUCTS, len(top_products_list)))


Country options count: 38
Using top N products for dropdown (N=50): 50


In [31]:
#build widgets and update function, display inline
country_dropdown = widgets.Dropdown(options=country_options, value='All', description='Country:', layout=widgets.Layout(width='48%'))
product_dropdown = widgets.Dropdown(options=product_options, value='All', description='Product:', layout=widgets.Layout(width='48%'))

out_time = widgets.Output()
out_top = widgets.Output()
out_hour = widgets.Output()

def update_plots(change=None):
    selected_country = country_dropdown.value
    selected_product = product_dropdown.value

    dff = df
    if selected_country != 'All' and 'Country' in dff.columns:
        dff = dff[dff['Country'] == selected_country]
    if selected_product != 'All':
        dff = dff[dff['Product'] == selected_product]

    # Sales over time (daily)
    times = dff.groupby('Date', as_index=False)['Sales'].sum().sort_values('Date')
    if times.empty:
        fig_time = px.line(x=[], y=[], title=f"Sales Over Time — no data for {selected_country} / {selected_product}")
    else:
        fig_time = px.line(times, x='Date', y='Sales', title=f"Sales Over Time — Country: {selected_country}  |  Product: {selected_product}")
        fig_time.update_xaxes(rangeslider_visible=True)
    fig_time.update_layout(height=360)

    # Top 5 products for the selected country (ignore product dropdown so user sees best sellers)
    if selected_country == 'All' or 'Country' not in df.columns:
        prod_source = df
    else:
        prod_source = df[df['Country'] == selected_country]
    top5 = prod_source.groupby('Product', as_index=False)['Sales'].sum().sort_values('Sales', ascending=False).head(5)
    if top5.empty:
        fig_top = px.bar(x=[], y=[], title="Top 5 Products — no data")
    else:
        fig_top = px.bar(top5, x='Product', y='Sales', title=f"Top 5 Products — Country: {selected_country}")
    fig_top.update_layout(height=360)

    # Sales by hour
    hour_data = dff.groupby('Hour', as_index=False)['Sales'].sum().sort_values('Hour')
    if hour_data.empty:
        fig_hour = px.bar(x=[], y=[], title="Sales by Hour — no data")
    else:
        fig_hour = px.bar(hour_data, x='Hour', y='Sales', title=f"Sales by Hour — {selected_country} / {selected_product}")
        fig_hour.update_layout(xaxis=dict(tickmode='linear'))
    fig_hour.update_layout(height=320)

    # Render
    with out_time:
        clear_output(wait=True)
        fig_time.show()
    with out_top:
        clear_output(wait=True)
        fig_top.show()
    with out_hour:
        clear_output(wait=True)
        fig_hour.show()

# Wire up observers
country_dropdown.observe(update_plots, names='value')
product_dropdown.observe(update_plots, names='value')

# Initial render and show controls
update_plots()
controls = widgets.HBox([country_dropdown, product_dropdown])
display(controls)
display(widgets.HBox([out_time, out_top]))
display(out_hour)


Output()

ValueError: Cannot accept list of column references or list of columns for both `x` and `y`.

In [33]:
#optional: export current three figures as a single HTML file
from plotly.offline import plot
selected_country = country_dropdown.value
selected_product = product_dropdown.value

dff = df
if selected_country != 'All' and 'Country' in dff.columns:
    dff = dff[dff['Country'] == selected_country]
if selected_product != 'All':
    dff = dff[dff['Product'] == selected_product]

times = dff.groupby('Date', as_index=False)['Sales'].sum().sort_values('Date')
fig_time = px.line(times, x='Date', y='Sales', title=f"Sales Over Time — {selected_country} / {selected_product}")
top5 = (df[df['Country'] == selected_country] if (selected_country != 'All' and 'Country' in df.columns) else df).groupby('Product', as_index=False)['Sales'].sum().sort_values('Sales', ascending=False).head(5)
fig_top = px.bar(top5, x='Product', y='Sales', title=f"Top 5 Products — {selected_country}")
hour_data = dff.groupby('Hour', as_index=False)['Sales'].sum().sort_values('Hour')
fig_hour = px.bar(hour_data, x='Hour', y='Sales', title=f"Sales by Hour — {selected_country} / {selected_product}")

html_filename = "online_retail_dashboard.html"
with open(html_filename, 'w', encoding='utf-8') as f:
    f.write("<html><head><meta charset='utf-8'></head><body>\n")
    f.write(plot(fig_time, include_plotlyjs='cdn', output_type='div'))
    f.write(plot(fig_top, include_plotlyjs=False, output_type='div'))
    f.write(plot(fig_hour, include_plotlyjs=False, output_type='div'))
    f.write("\n</body></html>")
print("Saved dashboard HTML to:", html_filename)
# To download in Colab UI: left Files pane or:
# from google.colab import files; files.download(html_filename)


Saved dashboard HTML to: online_retail_dashboard.html
